# Tutorial - Boolean Circuits

This tutorial covers:
- Low-level circuit construction (using `circkit` framework)
- Using existing circuits from the `wbkit` framework (AES-128)
- Generating pseudorandomness in circuits (`wbkit`), existing and custom
- Applying countermeasures (masking, shuffling) to circuits (`wbkit`)

## Low-level circuit construction (`circkit`)

In [ ]:
from circkit.boolean import OptBooleanCircuit as BooleanCircuit